## Install libraries

In [1]:
!pip install sentencepiece
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00


## Load model

In [2]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
os.environ["HF_ENDPOINT"] = "https://huggingface.co"
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Set the prompt

In [3]:
template = """You are expert in indian law.
Example 1:
Query: the appellant on february 9, 1961 was appointed as an officer in grade iii in the respondent bank ( for short'the bank'). he was promoted on april 1, 1968 to the grade officer in the foreign exchange department in the head office of the bank. sometime in 1964, mch society ( for short'the society') was formed of which the appellant was one of the chief promoters and thereafter its secretary. the object of the society was to construct residential premises for the employees of the bank and its other members. it appears that the complaint was received in respect of the affairs of the society relating to misappropriation of the funds of the society and consequently, in exercise of the powers under section s of act a1, the registrar on april 23, 1969 instituted an inquiry thereof. p1 was appointed the registrar's nominee who on october 4, 1969 ; submitted the report holding the appellant and two other office bearers of the society negligent in dealing with the funds of the society causing a loss to the tune of rs. 3, 59, 000 / -. the registrar on october 21, 1969, passed an order appointing an officer under section s of a1 to assess the loss caused to the society. however,
Docuemnt: whoever, being in any manner entrusted with property, or with any dominion over property in his capacity of a public servant or in the way of his business as a banker, merchant, factor, broker, attorney or agent, commits criminal breach of trust in respect of that property, shall be punished with 1 [ imprisonment for life ], or with imprisonment of either description for a term which may extend to ten years, and shall also be liable to fine. substituted by act 26 of 1955, section 117 and schedule, for " transportation for life " ( w. e. f. 1 - 1 - 1956 ).
Given the content of the Document, does it relevant to the Query? Yes

Example 2:
Query: the appellant on february 9, 1961 was appointed as an officer in grade iii in the respondent bank ( for short'the bank'). he was promoted on april 1, 1968 to the grade officer in the foreign exchange department in the head office of the bank. sometime in 1964, mch society ( for short'the society') was formed of which the appellant was one of the chief promoters and thereafter its secretary. the object of the society was to construct residential premises for the employees of the bank and its other members. it appears that the complaint was received in respect of the affairs of the society relating to misappropriation of the funds of the society and consequently, in exercise of the powers under section s of act a1, the registrar on april 23, 1969 instituted an inquiry thereof. p1 was appointed the registrar's nominee who on october 4, 1969 ; submitted the report holding the appellant and two other office bearers of the society negligent in dealing with the funds of the society causing a loss to the tune of rs. 3, 59, 000 / -. the registrar on october 21, 1969, passed an order appointing an officer under section s of a1 to assess the loss caused to the society. however,
Docuemnt: in this act, " dowry " means any property or valuable security given or agreed to be given either directly or indirectly - ( a ) by one party to a marriage to the other party to the marriage ; or ( b ) by the parents of either party to a marriage or by any other person, to either party to the marriage or to any other person ; at or before 1 [ or at any time after the marriage ] 2 [ in connection with the marriage of the said parties, but does not include ] dower or mahr in the case of persons to whom the muslim personal law ( shariat ) applies 3 [... ] explanation ii. - the expression " valuable security " has the same meaning as in section 30 of the indian penal code ( 45 of 1860 ). substituted by act 43 of 1986, section 2, for " or after the marriage " ( w. e. f. 19 - 11 - 1986 ). for the words " as consideration for the marriage of the said parties, but does not include ", substituted by dowry prohibition ( amendment ) act, 1984, section 2 ( a ) ( w. e. f. 2 - 10 - 1995 ). explanation i omitted by dowry prohibition ( amendment ) act, 1984
Given the content of the Document, does it relevant to the Query? No

Example 3:
{}
Given the content of the statute, does it relevant to the mentioned Legal Situation?
"""

### probability of token

In [4]:
def get_score(tokens):
  token_true_id  = tokenizer.get_vocab()['▁true']
  token_false_id = tokenizer.get_vocab()['▁false']
  import torch
  output = model.generate(
      tokens,
      max_new_tokens=1,
      return_dict_in_generate=True,
      output_scores=True,
  )
  batch_scores = output.scores[0]
  batch_scores = batch_scores[:, [token_false_id, token_true_id]]
  batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
  score = batch_scores[:, 1].exp().tolist()[0]
  return score

### Q_1, D1 candidate doc is relevant

In [5]:
new_input_content = """Query: having been selected by the public service commission, the respondent herein was appointed as law officer - cum - draftsman in the directorate of cooperation. there was only one post in the same cadre and it had no promotional avenues. he filed a representation that his post be upgraded or two promotional avenues be provided to him. several representations made by him having not received consideration at the hands of the appellants, the respondent herein filed a writ petition seeking for a specific direction upon the appellant herein to provide at least two promotional avenues. the said contention of the respondent was accepted by the high court and by reason of its impugned judgment the appellant was directed to provide'the graded scale'to the appellant by providing three grades, the initial being grade iii which is the post of law officer cum draftsman and thereafter grade ii and grade i. officer of judicial service. it was further directed : " " the scale of pay of grade ii law officer - cum - draftsman shall be same as grade - ii officer of the judicial service. the scale of pay of grade - i law officer - cum - draftsman shall be equal to the scale of pay of grade - i officer of judicial service. " " questioning
Document: ( 1 ) notwithstanding anything in article 32 every high court shall have powers, throughout the territories in relation to which it exercise jurisdiction, to issue to any person or authority, including in appropriate cases, any government, within those territories directions, orders or writs, including writs in the nature of habeas corpus, mandamus, prohibitions, quo warranto and certiorari, or any of them, for the enforcement of any of the rights conferred by part iii and for any other purpose ( 2 ) the power conferred by clause ( 1 ) to issue directions, orders or writs to any government, authority or person may also be exercised by any high court exercising jurisdiction in relation to the territories within which the cause of action, wholly or in part, arises for the exercise of such power, notwithstanding that the seat of such government or authority or the residence of such person is not within those territories ( 3 ) where any party against whom an interim order, whether by way of injunction or stay or in any other manner, is made on, or in any proceedings relating to, a petition under clause ( 1 ), without ( a ) furnishing to such party copies of such petition and all documents in support of the plea for such interim
"""
tokens = tokenizer(template.format(new_input_content), return_tensors="pt").input_ids
outputs = model.generate(tokens)
str_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(str_output)
score = get_score(tokens)
print(score)


Token indices sequence length is longer than the specified maximum sequence length for this model (1688 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


No
0.3771508038043976


### Q_1, D2 candidate doc is non-relevant and should achieve lower score compared to the above one!

In [6]:
new_input_content = """Query: having been selected by the public service commission, the respondent herein was appointed as law officer - cum - draftsman in the directorate of cooperation. there was only one post in the same cadre and it had no promotional avenues. he filed a representation that his post be upgraded or two promotional avenues be provided to him. several representations made by him having not received consideration at the hands of the appellants, the respondent herein filed a writ petition seeking for a specific direction upon the appellant herein to provide at least two promotional avenues. the said contention of the respondent was accepted by the high court and by reason of its impugned judgment the appellant was directed to provide'the graded scale'to the appellant by providing three grades, the initial being grade iii which is the post of law officer cum draftsman and thereafter grade ii and grade i. officer of judicial service. it was further directed : " " the scale of pay of grade ii law officer - cum - draftsman shall be same as grade - ii officer of the judicial service. the scale of pay of grade - i law officer - cum - draftsman shall be equal to the scale of pay of grade - i officer of judicial service. " " questioning
Document: ( 1 ) whenever it appears to the appropriate government that land in any locality is needed or is likely to be needed for any public purpose 1 [ or for a company ], a notification to that effect shall be published in the official gazette 2 [ and in two daily newspapers circulating in that locality of which at least one shall be in the regional language ] and the collector shall cause public notice of the substance of such notification to be given at convenient places in the said locality 3 [ the last of the dates of such publication and the giving of such public notice, being hereinafter referred to as the date of publication of the notification ]. ( 2 ) thereupon it shall be lawful for any officer, either, generally or specially authorised by such government in this behalf, and for his servants and workmen, to enter upon and survey and take levels of any land in such locality ; to dig or bore in the sub - soil ; to do all other acts necessary to ascertain whether the land is adapted for such purpose ; to set out the boundaries of the land proposed to be taken and the intended line of the work ( if any ) proposed to be made thereon ; to mark such levels, boundaries and line by placing marks and cutting trenches,
"""
tokens = tokenizer(template.format(new_input_content), return_tensors="pt").input_ids
outputs = model.generate(tokens)
str_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(str_output)
score = get_score(tokens)
print(score)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


No
0.36909031867980957


D1 score > D2 score. The model did a good relevance ordering at least for these two examples


# Try to assess the effectiveness on the validation set (queries_val.tsv)


## downloading files

In [ ]:
!wget https://www.dropbox.com/scl/fi/md1qj0dz07wi66lan1aah/afternoon_session_files.zip?rlkey=ficet9hbs55wxs7e11u6yi9w0&dl=0
!unzip afternoon_session_files.zip?rlkey=ficet9hbs55wxs7e11u6yi9w0&dl=0

## Steps
1. Iterate on each query
2. Per candidate document for that query, compute the score
3. Create ranking run file in TREC format
4. Evalute the effectiveness
5. Each one can present their results and how they designed and selected LLM at the end and try to improve it in future.
Note:

- Using larger LLM takes more time but probably lead to higher effectiveness so you can take this into the account.

- For loading larger model with less gpu you can take a look on https://huggingface.co/blog/4bit-transformers-bitsandbytes for maybe experimenting with larger models for friday.

- Feel free to select another LLM based on the leaderboard: https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard

- Feel free to change the prompt

- Feel free to use the utils code and implementations from previous notebook


